In [1]:
%cd /hdd/yuchen

/hdd/yuchen


In [2]:
from PIL import Image
import torch
from torch import nn, optim
import glob
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, BatchSampler
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import random
import nltk, re, string, collections
import scipy 
import IProgress 
from ipywidgets import FloatProgress
from nilearn import connectome
from sklearn.preprocessing import Normalizer, OrdinalEncoder, OneHotEncoder, StandardScaler
from torch.nn import Linear, ReLU, Dropout
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling, BatchNorm, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from matplotlib import pyplot as plt

%matplotlib inline

In [3]:
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.cuda.empty_cache()

In [4]:
featname = 'init_latent'
use_stim = 'each'

subject= 'subj01'
topdir = '/hdd/yuchen/nsdfeat'

savedir = f'{topdir}/subjfeat/'
featdir = f'{topdir}/15dim/{featname}/'

nsd_expdesign = scipy.io.loadmat('/hdd/yuchen/nsd/nsddata/experiments/nsd/nsd_expdesign.mat')

# Note that most of them are 1-base index!
sharedix = nsd_expdesign['sharedix'] -1 
stims = np.load(f'/hdd/yuchen/mrifeat/{subject}/{subject}_stims.npy')

feats = []
tr_idx = np.zeros(len(stims))

ct = 0

for idx, s in tqdm(enumerate(stims)): 
    if s in sharedix:
        tr_idx[idx] = 0
    else:
        tr_idx[idx] = 1    
    if os.path.exists(f'{featdir}/{s:06}.npy'):
        feat = np.load(f'{featdir}/{s:06}.npy')
        feats.append(feat)
    else: 
        ct += 1

feats = np.stack(feats)    

feats_tr = feats[tr_idx==1]
feats_te = feats[tr_idx==0]

0it [00:00, ?it/s]

In [5]:
savedir = '/hdd/yuchen/mrifeat/subj01/'

betas_tr = np.load(f'{savedir}/{subject}_early_betas_tr.npy')
betas_te = np.load(f'{savedir}/{subject}_early_betas_te.npy')
betas_tr_avg = np.load(f'{savedir}/{subject}_early_betas_ave_tr.npy')
betas_te_avg = np.load(f'{savedir}/{subject}_early_betas_ave_te.npy')


In [6]:
stims = np.load(f'/hdd/yuchen/mrifeat/{subject}/{subject}_stims_ave.npy')

feats_avg = []
tr_idx = np.zeros(len(stims))

ct = 0

for idx, s in tqdm(enumerate(stims)): 
    if s in sharedix:
        tr_idx[idx] = 0
    else:
        tr_idx[idx] = 1    
    if os.path.exists(f'{featdir}/{s:06}.npy'):
        # feat = np.load(f'{featdir}/{s:06}.npy')[0]
        feat = np.load(f'{featdir}/{s:06}.npy')
        
        feats_avg.append(feat)
    else: 
        # print(f'{s:06}.npy does not exist')
        # feats_avg.append('empty')
        feats_avg.append(np.array([0]*768*15))
        ct += 1

feats_avg = np.stack(feats_avg)    

feats_tr_avg = feats_avg[tr_idx==1]
feats_te_avg = feats_avg[tr_idx==0]

0it [00:00, ?it/s]

In [7]:
print('feats_tr', feats_tr.shape)
print('feats_te', feats_te.shape)
print('betas_tr', betas_tr.shape)
print('betas_te', betas_te.shape)

print('feats_tr_avg', feats_tr_avg.shape)
print('feats_te_avg', feats_te_avg.shape)
print('betas_tr_avg', betas_tr_avg.shape)
print('betas_te_avg', betas_te_avg.shape)


feats_tr (24980, 6400)
feats_te (2770, 6400)
betas_tr (24980, 5917)
betas_te (2770, 5917)
feats_tr_avg (8859, 6400)
feats_te_avg (982, 6400)
betas_tr_avg (8859, 5917)
betas_te_avg (982, 5917)


In [8]:
betas_tr_ = (betas_tr - betas_tr.mean()) / betas_tr.std()
betas_tr_avg_ = (betas_tr_avg - betas_tr_avg.mean()) / betas_tr_avg.std()
betas_te_ = (betas_te - betas_te.mean()) / betas_te.std()
betas_te_avg_ = (betas_te_avg - betas_te_avg.mean()) / betas_te_avg.std()

## goal: 0.61

In [7]:
import argparse, os
import numpy as np
from himalaya.backend import set_backend
from himalaya.ridge import RidgeCV
from himalaya.scoring import correlation_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
alpha = [0.000001,0.00001,0.0001,0.001,0.01, 0.1, 1]

ridge = RidgeCV(alphas=alpha)

preprocess_pipeline = make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
    )
pipeline = make_pipeline(
        preprocess_pipeline,
        ridge,
)    

In [8]:
X = betas_tr
Y = feats_tr
X_te = betas_te_avg
pipeline.fit(X, Y)
scores = pipeline.predict(X_te)
from sklearn.metrics import mean_squared_error
mean_squared_error(scores,feats_te_avg)

0.6132555970819675

## gcn best: 0.55675

In [23]:
num_roi = 4

num_neurons = betas_tr_.shape[-1]
num_pad = (num_neurons // num_roi + 1) * num_roi - num_neurons

betas_tr = np.pad(betas_tr_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_tr_avg = np.pad(betas_tr_avg_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_te = np.pad(betas_te_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_te_avg = np.pad(betas_te_avg_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)

In [24]:
num_neurons_each_roi = int(betas_te_avg.shape[-1]/num_roi)

conn_measure = connectome.ConnectivityMeasure(kind='correlation')

function_con = betas_tr.reshape(-1, num_roi, num_neurons_each_roi)

connectivity = conn_measure.fit_transform([np.mean(function_con, axis=-1)])
for i in range(len(connectivity[0])):
    connectivity[0][i, i] = 0  # setting the i-th row to zero

/hdd/yuchen/anaconda3/envs/ldm/lib/python3.8/site-packages/nilearn/connectome/connectivity_matrices.py:509: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  covariances_std = [


In [25]:
connectivity.shape

(1, 4, 4)

In [63]:
adjmatrix = np.where(abs(connectivity[0]) > 0.8)
edge_index = np.array(adjmatrix)  # edge index matrix of shape [2, num_edges]

edge_index.shape

(2, 6)

In [64]:
edge_attr = []
for idx in range(edge_index.shape[1]):
    edge_attr.append(connectivity[0][edge_index[0,idx], edge_index[1,idx]])
edge_attr = np.array(edge_attr)
edge_attr.shape

(6,)

In [72]:
edge_index

array([[0, 1, 1, 2, 2, 3],
       [1, 0, 2, 1, 3, 2]])

In [65]:
num_trial_tr, num_trials_te = feats_tr.shape[0], feats_te_avg.shape[0]

train_dataset = []

for i in range(num_trial_tr):
    train_dataset.append(Data(x=torch.tensor(betas_tr.reshape(-1, num_roi, num_neurons_each_roi)[i], dtype=torch.float), 
            edge_index=torch.tensor(edge_index, dtype=torch.long), 
            edge_attr=torch.tensor(edge_attr, dtype=torch.float),
            y=torch.tensor(feats_tr[i], dtype=torch.float)
            ))
    
test_dataset = []
for i in range(num_trials_te):
    test_dataset.append(Data(x=torch.tensor(betas_te_avg.reshape(-1, num_roi, num_neurons_each_roi)[i], dtype=torch.float), 
            edge_index=torch.tensor(edge_index, dtype=torch.long), 
            edge_attr=torch.tensor(edge_attr, dtype=torch.float),
            y=torch.tensor(feats_te_avg[i], dtype=torch.float)
            ))

In [66]:
len(train_dataset), len(test_dataset)

(24980, 982)

In [67]:
train_dataset[0], test_dataset[0]

(Data(x=[4, 1480], edge_index=[2, 6], edge_attr=[6], y=[6400]),
 Data(x=[4, 1480], edge_index=[2, 6], edge_attr=[6], y=[6400]))

In [68]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [69]:
from torch_geometric.nn import GCNConv, TopKPooling, BatchNorm,ChebConv, SAGEConv,GATConv
from torch_geometric.nn import global_mean_pool, global_max_pool,global_add_pool

class GNN(torch.nn.Module):
    def __init__(self, num_neurons_each_roi):
        super(GNN, self).__init__()
        
        self.conv1 = ChebConv(num_neurons_each_roi, 256, K=4)

        self.conv = ChebConv(256, 256, K=3)
        self.bn = BatchNorm(256)

        nnconv_channel = 16
        self.nnconv = nn.Sequential(
            nn.Conv1d(1, nnconv_channel, kernel_size=1),
            nn.LeakyReLU(),
        )
        self.net = nn.Sequential(
            nn.Conv1d(nnconv_channel, nnconv_channel, kernel_size=3, padding = 1),
            nn.LeakyReLU(),
        )
        self.downsample = nn.Conv1d(nnconv_channel, 1, kernel_size=1)

        self.fc = nn.Sequential(
            nn.Linear(256, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 6400)
        )

    def forward(self, data):
        x, edge_index, batch,edge_attr = data.x, data.edge_index, data.batch, data.edge_attr
        batch_size = batch.max() + 1

        x = F.relu(self.conv1(x, edge_index, edge_attr))
        
        for i in range(1):
            a = self.conv(x, edge_index, edge_attr)
            a = self.bn(a)
            x = x + F.leaky_relu(a)
        
        x = global_mean_pool(x, batch) # print(x.shape) torch.Size([64, 256])

        x = x.unsqueeze(1)
        x = self.nnconv(x)
        for j in range(2):
            x = x + self.net(x)
        x = self.downsample(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        
        return x

In [70]:
device = 'cuda:1'
model = GCNWithPooling(num_neurons_each_roi=num_neurons_each_roi).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
patience = 5
ct = 0
best_loss_val = np.inf

for epoch in range(num_epochs):
    total_loss_train = 0
    model.train()
    for batch in train_dataloader:
        batch = batch.to(device)
        batch_num = batch.batch.max() + 1
        feats = batch['y'].reshape(batch_num,-1).to(device)
        pred = model(batch)
        
        loss = loss_fn(pred, feats)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss_train += loss.item()
    print('[epoch {}] train loss {}'.format(epoch, round(total_loss_train/len(train_dataloader), 5)), end = ' ')

    model.eval()
    total_loss_val = 0
    with torch.no_grad():
        for batch in test_dataloader:
            batch = batch.to(device)
            batch_num = batch.batch.max() + 1
            feats = batch['y'].reshape(batch_num,-1).to(device)
            pred = model(batch)
            loss = loss_fn(pred, feats)
            total_loss_val += loss.item()
        total_loss_val = total_loss_val/len(test_dataloader)
        print('val loss {}'.format(round(total_loss_val, 5)))
        
        if total_loss_val < best_loss_val:
            ct = 0
            best_loss_val = total_loss_val
            torch.save(model.state_dict(), '/hdd/yuchen/modelweights_visual_gcn_testesttes.pth')
        else:
            ct += 1
            if ct > patience:
                print('end')
                break

[epoch 0] train loss 0.62194 val loss 0.59383
[epoch 1] train loss 0.60174 val loss 0.57447
[epoch 2] train loss 0.58985 val loss 0.57166
[epoch 3] train loss 0.58079 val loss 0.56628
[epoch 4] train loss 0.57268 val loss 0.56573
[epoch 5] train loss 0.56579 val loss 0.56229
[epoch 6] train loss 0.55928 val loss 0.56794
[epoch 7] train loss 0.5528 val loss 0.56381
[epoch 8] train loss 0.5473 val loss 0.56882
[epoch 9] train loss 0.54206 val loss 0.56724
[epoch 10] train loss 0.53653 val loss 0.55819
[epoch 11] train loss 0.53144 val loss 0.56191
[epoch 12] train loss 0.52694 val loss 0.56415
[epoch 13] train loss 0.52203 val loss 0.56767
[epoch 14] train loss 0.51766 val loss 0.56789
[epoch 15] train loss 0.51352 val loss 0.5662
[epoch 16] train loss 0.5094 val loss 0.56555
end


num_roi=8: 0.5654079392552376
num_roi=16: 0.5733139477670193
num_roi=4: 0.56504
    conv512: 0.5668724402785301
    conv256: 0.5616156719624996
        linear256->6400: 0.5727455839514732
        GCNimproved=True: 0.562708642333746
        linear256->256->6400: 0.5671156793832779
        linear256->1024->6400: 0.5647310167551041
        linear256->512->1024->6400: 0.5680358745157719
        linear256->512->512->6400: 0.5688614509999752
        ChebConv(K=2) 0.5641; (K=3) 0.5607; (K=4) 0.5610; (K=3&4) 0.5597; (K=4&3) 0.5596
        SAGEConv and GATConv: worse 
            nnconv.T: 0.5561986267566681 (2layers of nnconv.T is worse)
            nnconv: 2skip: 0.5567543022334576 (0.5583899803459644)
                replace relu with leakyrelu: 0.557660523802042
                replace nnconv's relu with leakyrelu: 0.5604802779853344
                replace gcn's second and all nnconv's relu with leakyrelu: 0.5566234886646271
 

In [19]:
device = 'cuda:1'
model = GCNWithPooling(num_neurons_each_roi=num_neurons_each_roi).to(device)
model.load_state_dict(torch.load('/hdd/yuchen/modelweights_visual_gcn.pth'))

<All keys matched successfully>

In [24]:
model.eval()
lst = []
lst_gt = []

with torch.no_grad():
    for batch in test_dataloader:
        batch = batch.to(device)
        batch_num = batch.batch.max() + 1
        pred = model(batch)
        batch_num = batch.batch.max() + 1
        feats = batch['y'].reshape(batch_num,-1).to(device)
        
        lst.append(pred.cpu().detach().numpy())
        lst_gt.append(feats.cpu().detach().numpy())

In [25]:
lst = np.vstack(lst)
lst_gt = np.vstack(lst_gt)

In [26]:
from sklearn.metrics import mean_squared_error
mean_squared_error(lst,lst_gt)

0.55870265

In [27]:
np.save('/hdd/yuchen/temp_lst_latent_init.npy', lst)

## adapted text encoder best: 0.56112

In [22]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, betas, feats):
        self.betas = torch.tensor(betas, dtype=torch.float32)
        self.feats = torch.tensor(feats, dtype=torch.float32)

    def __len__(self):
        return len(self.betas)

    def __getitem__(self, idx):
        return {'betas': self.betas[idx], 'feats': self.feats[idx]}

train_dataset = CustomDataset(betas_tr_, feats_tr)
test_dataset = CustomDataset(betas_te_avg_, feats_te_avg)

In [23]:
train_dataset[0]['betas'].shape, train_dataset[0]['feats'].shape

(torch.Size([5917]), torch.Size([6400]))

In [24]:
batch_size = 256  
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for batch in train_dataloader:
    break
num_neurons = batch['betas'].shape[-1]
num_features = batch['feats'].shape[-1]

In [89]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self, num_neurons, num_features):
        super(NeuralNetwork, self).__init__()
        
        self.num_neurons = num_neurons
        channel = 64
        self.conv1d = nn.Conv1d(1, channel, kernel_size=1)
        self.relu = nn.ReLU()
        self.net = nn.Sequential(
            nn.Conv1d(channel, channel, kernel_size=11, padding = 5),
            nn.ReLU(),
            nn.Conv1d(channel, channel, kernel_size=11, padding = 5),
            nn.ReLU(),
        )
        self.downsample = nn.Conv1d(channel, 1, kernel_size=1)

        self.fc = nn.Sequential(
            nn.Linear(num_neurons, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 6400)
        )

    def forward(self, x):
        x = self.conv1d(x)
        for i in range(2):
            x = x + self.net(x)
        x = self.downsample(x)
        x = x.reshape(-1, self.num_neurons)
        x = self.fc(x)
        return x

In [90]:
device = 'cuda:1'
model = NeuralNetwork(num_neurons = num_neurons, num_features = num_features).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

num_epochs = 100
patience = 5
ct = 0
best_loss_val = np.inf

for epoch in range(num_epochs):
    total_loss_train = 0
    model.train()
    for batch in train_dataloader:
        betas = batch['betas'].to(device).unsqueeze(1)
        feats = batch['feats'].to(device)
        
        pred = model(betas)
        loss = loss_fn(pred, feats)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss_train += loss.item()
    print('[epoch {}] train loss {}'.format(epoch, round(total_loss_train/len(train_dataloader), 5)), end = ' ')

    model.eval()
    total_loss_val = 0
    with torch.no_grad():
        for batch in test_dataloader:
            betas = batch['betas'].to(device).unsqueeze(1)
            feats = batch['feats'].to(device)
            pred = model(betas)
            loss = loss_fn(pred, feats)
            total_loss_val += loss.item()
        total_loss_val = total_loss_val/len(test_dataloader)
        print('val loss {}'.format(round(total_loss_val, 5)))
        
        if total_loss_val < best_loss_val:
            ct = 0
            best_loss_val = total_loss_val
            torch.save(model.state_dict(), '/hdd/yuchen/modelweights_visual_textnn.pth')
        else:
            ct += 1
            if ct > patience:
                print('end')
                break

[epoch 0] train loss 0.63809 val loss 0.59885
[epoch 1] train loss 0.61257 val loss 0.58145
[epoch 2] train loss 0.59541 val loss 0.5702
[epoch 3] train loss 0.58071 val loss 0.56452
[epoch 4] train loss 0.56847 val loss 0.56616
[epoch 5] train loss 0.55726 val loss 0.56186
[epoch 6] train loss 0.54642 val loss 0.56161
[epoch 7] train loss 0.53635 val loss 0.56176
[epoch 8] train loss 0.52604 val loss 0.5662
[epoch 9] train loss 0.51801 val loss 0.56397
[epoch 10] train loss 0.50963 val loss 0.566
[epoch 11] train loss 0.50274 val loss 0.56795
[epoch 12] train loss 0.49649 val loss 0.5684
end


In [91]:
best_loss_val

0.5616071820259094

## VAE best: 0.5686

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, betas, feats):
        self.betas = torch.tensor(betas, dtype=torch.float32)
        self.feats = torch.tensor(feats, dtype=torch.float32)

    def __len__(self):
        return len(self.betas)

    def __getitem__(self, idx):
        return {'betas': self.betas[idx], 'feats': self.feats[idx]}

train_dataset = CustomDataset(betas_tr_, feats_tr)
test_dataset = CustomDataset(betas_te_avg_, feats_te_avg)

In [10]:
batch_size = 256  
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for batch in train_dataloader:
    break
num_neurons = batch['betas'].shape[-1]
num_features = batch['feats'].shape[-1]

In [104]:
import torch
from torch import nn
from torch.nn import functional as F

class FBL(nn.Module):
    def __init__(self, in_features, out_features):
        super(FBL, self).__init__()
        self.fc = nn.Linear(in_features, out_features)
        self.bn = nn.BatchNorm1d(out_features)
        self.act = nn.LeakyReLU()

    def forward(self, x):
        x = self.fc(x)
        x = self.bn(x)
        return self.act(x)

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dims, z_dim):
        super(Encoder, self).__init__()
        self.ln1 = nn.Linear(input_dim, 512)
        self.relu = nn.LeakyReLU()
        self.ln2 = nn.Linear(512, 256)
        self.fc_mu = nn.Linear(256, z_dim)
        self.fc_log_var = nn.Linear(256, z_dim)

    def forward(self, x):
        x = self.ln1(x)
        x = self.relu(x)
        x = self.ln2(x)
        mu = self.fc_mu(x)
        log_var = self.fc_log_var(x)
        return mu, log_var

class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_dims, output_dim):
        super(Decoder, self).__init__()
        self.fbl1 = FBL(z_dim, hidden_dims)
        self.fbl2 = FBL(hidden_dims, hidden_dims)
        self.fbl3 = FBL(hidden_dims, hidden_dims)
        self.final_fc = nn.Linear(hidden_dims, output_dim)

    def forward(self, z):
        z = self.fbl1(z)
        for i in range(1):
            z = self.fbl2(z)
        z = self.fbl3(z)
        return self.final_fc(z)

class VAE(nn.Module):
    def __init__(self, fMRI_input_dim, hidden_dims, z_dim, output_dim):
        super(VAE, self).__init__()
        self.fMRI_encoder = Encoder(fMRI_input_dim, hidden_dims, z_dim)
        self.decoder = Decoder(z_dim, hidden_dims, output_dim)

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, fMRI):
        fMRI_mu, fMRI_log_var = self.fMRI_encoder(fMRI)
        z_fMRI = self.reparameterize(fMRI_mu, fMRI_log_var)
        fMRI_embedding = self.decoder(z_fMRI)

        return fMRI_embedding, fMRI_mu, fMRI_log_var, z_fMRI


In [105]:
device = 'cuda:1'
fMRI_input_dim = num_neurons  
hidden_dims = 4096 
z_dim = num_features  
output_dim = num_neurons  

model = BrainCLIPVAE(fMRI_input_dim, hidden_dims, z_dim, output_dim).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

num_epochs = 100
patience = 5
ct = 0
best_loss_val = np.inf

for epoch in range(num_epochs):
    total_loss_train = 0
    model.train()
    for batch in train_dataloader:
        betas = batch['betas'].to(device)
        feats = batch['feats'].to(device)

        fMRI_embedding, fMRI_mu, fMRI_log_var, z_fMRI = model(betas)
        
        recon_loss_fMRI = loss_fn(fMRI_embedding, betas)
        # kld_fMRI = -0.5 * torch.mean(1 + fMRI_log_var - fMRI_mu.pow(2) - fMRI_log_var.exp())
        kld_fMRI = loss_fn(feats, z_fMRI)
        loss = recon_loss_fMRI + kld_fMRI
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss_train += loss.item()
    print('[epoch {}] train loss {}'.format(epoch, round(total_loss_train/len(train_dataloader), 5)), end = ' ')
        
        
    model.eval()
    total_loss_val = 0
    with torch.no_grad():
        for batch in test_dataloader:
            betas = batch['betas'].to(device)
            feats = batch['feats'].to(device)

            fMRI_embedding, fMRI_mu, fMRI_log_var, z_fMRI = model(betas)
        
            loss = loss_fn(feats, z_fMRI)
            total_loss_val += loss.item()
        
        total_loss_val = total_loss_val/len(test_dataloader)
        print('val loss {}'.format(round(total_loss_val, 5)))
        
        if total_loss_val < best_loss_val:
            ct = 0
            best_loss_val = total_loss_val
            torch.save(model.state_dict(), '/hdd/yuchen/modelweights_visual_vae.pth')
        else:
            ct += 1
            if ct > patience:
                print('end')
                break

[epoch 0] train loss 1.23374 val loss 0.62704
[epoch 1] train loss 0.96317 val loss 0.61449
[epoch 2] train loss 0.91064 val loss 0.60464
[epoch 3] train loss 0.87798 val loss 0.60207
[epoch 4] train loss 0.85502 val loss 0.5979
[epoch 5] train loss 0.84282 val loss 0.58585
[epoch 6] train loss 0.82991 val loss 0.5885
[epoch 7] train loss 0.81926 val loss 0.57799
[epoch 8] train loss 0.81258 val loss 0.57824
[epoch 9] train loss 0.80211 val loss 0.58638
[epoch 10] train loss 0.79698 val loss 0.5876
[epoch 11] train loss 0.78941 val loss 0.57724
[epoch 12] train loss 0.78139 val loss 0.5731
[epoch 13] train loss 0.7754 val loss 0.5746
[epoch 14] train loss 0.76899 val loss 0.56979
[epoch 15] train loss 0.76398 val loss 0.56985
[epoch 16] train loss 0.75943 val loss 0.56863
[epoch 17] train loss 0.75712 val loss 0.57489
[epoch 18] train loss 0.74756 val loss 0.57048
[epoch 19] train loss 0.74389 val loss 0.57015
[epoch 20] train loss 0.73893 val loss 0.56874
[epoch 21] train loss 0.736 v

In [106]:
best_loss_val

0.5686286985874176

## use ventral stream data: regression: 0.697; gcn: 0.611

In [10]:
import argparse, os
import numpy as np
from himalaya.backend import set_backend
from himalaya.ridge import RidgeCV
from himalaya.scoring import correlation_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
alpha = [0.000001,0.00001,0.0001,0.001,0.01, 0.1, 1]

ridge = RidgeCV(alphas=alpha)

preprocess_pipeline = make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
    )
pipeline = make_pipeline(
        preprocess_pipeline,
        ridge,
)    

X = betas_tr
Y = feats_tr
X_te = betas_te_avg
pipeline.fit(X, Y)
scores = pipeline.predict(X_te)
from sklearn.metrics import mean_squared_error
mean_squared_error(scores,feats_te_avg)

0.6971274161290577

In [11]:
num_roi = 4
num_neurons = betas_tr_.shape[-1]
num_pad = (num_neurons // num_roi + 1) * num_roi - num_neurons

betas_tr = np.pad(betas_tr_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_tr_avg = np.pad(betas_tr_avg_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_te = np.pad(betas_te_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_te_avg = np.pad(betas_te_avg_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)

num_neurons_each_roi = int(betas_te_avg.shape[-1]/num_roi)
conn_measure = connectome.ConnectivityMeasure(kind='correlation')
function_con = betas_tr.reshape(-1, num_roi, num_neurons_each_roi)
connectivity = conn_measure.fit_transform([np.mean(function_con, axis=-1)])
for i in range(len(connectivity[0])):
    connectivity[0][i, i] = 0  # Setting the i-th row to zero

/hdd/yuchen/anaconda3/envs/ldm/lib/python3.8/site-packages/nilearn/connectome/connectivity_matrices.py:509: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  covariances_std = [


In [14]:
adjmatrix = np.where(abs(connectivity[0]) > 0.8)
edge_index = np.array(adjmatrix)  # Edge index matrix of shape [2, num_edges]
print(edge_index.shape)

edge_attr = []
for idx in range(edge_index.shape[1]):
    edge_attr.append(connectivity[0][edge_index[0,idx], edge_index[1,idx]])
edge_attr = np.array(edge_attr)

(2, 4)


In [15]:
num_trial_tr, num_trials_te = feats_tr.shape[0], feats_te_avg.shape[0]

train_dataset = []
for i in range(num_trial_tr):
    train_dataset.append(Data(x=torch.tensor(betas_tr.reshape(-1, num_roi, num_neurons_each_roi)[i], dtype=torch.float), 
            edge_index=torch.tensor(edge_index, dtype=torch.long), 
            edge_attr=torch.tensor(edge_attr, dtype=torch.float),
            y=torch.tensor(feats_tr[i], dtype=torch.float)
            ))
    
test_dataset = []
for i in range(num_trials_te):
    test_dataset.append(Data(x=torch.tensor(betas_te_avg.reshape(-1, num_roi, num_neurons_each_roi)[i], dtype=torch.float), 
            edge_index=torch.tensor(edge_index, dtype=torch.long), 
            edge_attr=torch.tensor(edge_attr, dtype=torch.float),
            y=torch.tensor(feats_te_avg[i], dtype=torch.float)
            ))

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [16]:
from torch_geometric.nn import GCNConv, TopKPooling, BatchNorm,ChebConv, SAGEConv,GATConv
from torch_geometric.nn import global_mean_pool, global_max_pool,global_add_pool

class GCNWithPooling(torch.nn.Module):
    def __init__(self, num_neurons_each_roi):
        super(GCNWithPooling, self).__init__()
        
        self.conv1 = ChebConv(num_neurons_each_roi, 256, K=4)

        self.dropout = nn.Dropout(p=0.1)
        self.conv = ChebConv(256, 256, K=3)
        self.bn = BatchNorm(256)

        nnconv_channel = 16
        self.nnconv = nn.Sequential(
            nn.Conv1d(1, nnconv_channel, kernel_size=1),
            nn.ReLU(), 
        )
        self.net = nn.Sequential(
            nn.Conv1d(nnconv_channel, nnconv_channel, kernel_size=3, padding = 1),
            nn.ReLU(),
        )
        self.downsample = nn.Conv1d(nnconv_channel, 1, kernel_size=1)

        self.fc = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 6400)
        )

    def forward(self, data):
        x, edge_index, batch,edge_attr = data.x, data.edge_index, data.batch, data.edge_attr
        batch_size = batch.max() + 1

        x = F.relu(self.conv1(x, edge_index, edge_attr))

        for i in range(1):
            a = self.conv(x, edge_index, edge_attr)
            a = self.bn(a)
            x = x + F.relu(a)

        x = global_mean_pool(x, batch) # print(x.shape) torch.Size([64, 256])

        x = x.unsqueeze(1)
        x = self.nnconv(x)
        
        for j in range(2):
            x = x + self.net(x)
        x = self.downsample(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        
        return x

In [19]:
device = 'cuda:1'
model = GCNWithPooling(num_neurons_each_roi=num_neurons_each_roi).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

num_epochs = 100
patience = 5
ct = 0
best_loss_val = np.inf

for epoch in range(num_epochs):
    total_loss_train = 0
    model.train()
    for batch in train_dataloader:
        batch = batch.to(device)
        batch_num = batch.batch.max() + 1
        feats = batch['y'].reshape(batch_num,-1).to(device)
        pred = model(batch)
        
        loss = loss_fn(pred, feats)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss_train += loss.item()
    print('[epoch {}] train loss {}'.format(epoch, round(total_loss_train/len(train_dataloader), 5)), end = ' ')

    model.eval()
    total_loss_val = 0
    with torch.no_grad():
        for batch in test_dataloader:
            batch = batch.to(device)
            batch_num = batch.batch.max() + 1
            feats = batch['y'].reshape(batch_num,-1).to(device)
            pred = model(batch)
            loss = loss_fn(pred, feats)
            total_loss_val += loss.item()
        total_loss_val = total_loss_val/len(test_dataloader)
        print('val loss {}'.format(round(total_loss_val, 5)))
        
        if total_loss_val < best_loss_val:
            ct = 0
            best_loss_val = total_loss_val
            torch.save(model.state_dict(), '/hdd/yuchen/modelweights_visual_gcn_test.pth')
        else:
            ct += 1
            if ct > patience:
                print('end')
                break

[epoch 0] train loss 0.65145 val loss 0.62354
[epoch 1] train loss 0.64557 val loss 0.62117
[epoch 2] train loss 0.64482 val loss 0.62083
[epoch 3] train loss 0.64456 val loss 0.62064
[epoch 4] train loss 0.64436 val loss 0.62031
[epoch 5] train loss 0.64402 val loss 0.61988
[epoch 6] train loss 0.64311 val loss 0.61861
[epoch 7] train loss 0.64154 val loss 0.6169
[epoch 8] train loss 0.63933 val loss 0.61434
[epoch 9] train loss 0.63718 val loss 0.61329
[epoch 10] train loss 0.63523 val loss 0.61178
[epoch 11] train loss 0.63353 val loss 0.61143
[epoch 12] train loss 0.63188 val loss 0.61121
[epoch 13] train loss 0.63019 val loss 0.61096
[epoch 14] train loss 0.62867 val loss 0.61115
[epoch 15] train loss 0.62739 val loss 0.611
[epoch 16] train loss 0.62583 val loss 0.6112
[epoch 17] train loss 0.62411 val loss 0.61157
[epoch 18] train loss 0.6224 val loss 0.61154
[epoch 19] train loss 0.62063 val loss 0.61303
end


In [181]:
out = model(batch)
out.shape

torch.Size([16, 6400])

In [151]:
model(dataset[0]).shape

torch.Size([1480, 64])

In [146]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [169]:
out = model(batch)
out.reshape(16, -1, 64).shape

torch.Size([16, 1480, 64])

In [140]:
train_loader = DataLoader(dataset, batch_size=1)
len(train_loader)

100

In [148]:
from torch_geometric.loader import DataLoader

In [53]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

# Load a dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

In [54]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [55]:
data.num_classes

AttributeError: 'GlobalStorage' object has no attribute 'num_classes'